# Regressionanalyse Youtube Zielvariable Views

LAden der Pickle-Objekte aus der Datenvorverarbeitung

In [ ]:

# Zelle 1: Setup und Imports
import pandas as pd
import numpy as np
import pickle
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

# Plotstil festlegen (verwenden einen basic matplotlib style)
plt.style.use('default')
# Seaborn Plotting Style setzen
sns.set_theme(style="whitegrid")

%matplotlib inline

plt.rcParams['font.family'] = 'DejaVu Sans'  # Alternativ: 'Liberation Sans', 'Arial Unicode MS'

# Display-Optionen
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Zelle 2: Laden der vorverarbeiteten Daten
# Laden der Pickle-Datei
with open('../data/processed/processed_data.pkl', 'rb') as f:
    data = pickle.load(f)

# Extrahieren der einzelnen Komponenten
X_train = data['X_train']
X_val = data['X_val']
X_test = data['X_test']
y_train = data['y_train']
y_val = data['y_val']
y_test = data['y_test']
scaler = data['scaler']
feature_columns = data['feature_columns']
target_columns = data['target_columns']

# Überprüfung der geladenen Daten
print("Daten erfolgreich geladen!")
print(f"\nTrainings-Set Dimensionen:")
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape}")
print(f"\nFeature-Spalten:")
print(feature_columns)
print(f"\nZiel-Spalten:")
print(target_columns)

# Zelle 3: Kurze Dateninspektion
# Überblick über die Feature-Verteilungen
print("\nStatistische Kennzahlen der Features im Trainingsset:")
print(X_train.describe())

# Überblick über die Zielvariblen
print("\nStatistische Kennzahlen der Zielvariablen im Trainingsset:")
print(y_train.describe())

# Visualisierung der Verteilung der Zielvariablen
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
fig.suptitle('Verteilung der Zielvariablen')

for i, col in enumerate(target_columns):
    sns.histplot(data=y_train, x=col, ax=axes[i])
    axes[i].set_title(col)
    
plt.tight_layout()
plt.show()

### Funktionsdefinition für Analyse der Prädiktoren

In [3]:
def analyze_features_for_target(X_train, y_train, target_name, feature_columns):
    """
    Führt eine umfassende Feature-Analyse für eine Zielvariable durch
    """
    from sklearn.feature_selection import SelectKBest, f_regression
    from sklearn.linear_model import Lasso
    from sklearn.impute import SimpleImputer
    import pandas as pd
    import numpy as np
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    # Nur numerische Spalten auswählen
    numeric_features = X_train.select_dtypes(include=['float64', 'int64']).columns
    X_numeric = X_train[numeric_features]
    
    # Imputation der fehlenden Werte
    imputer = SimpleImputer(strategy='mean')
    X_numeric_imputed = pd.DataFrame(
        imputer.fit_transform(X_numeric),
        columns=X_numeric.columns
    )
    
    target = y_train[target_name]
    results = {}
    
    # 1. F-Regression
    selector = SelectKBest(score_func=f_regression, k='all')
    selector.fit(X_numeric_imputed, target)
    
    f_scores = pd.DataFrame({
        'Feature': numeric_features,
        'F_Score': selector.scores_,
        'P_Value': selector.pvalues_
    }).sort_values('F_Score', ascending=False)
    
    results['f_scores'] = f_scores
    
    # 2. Lasso
    lasso = Lasso(alpha=0.01)
    lasso.fit(X_numeric_imputed, target)
    
    lasso_importance = pd.DataFrame({
        'Feature': numeric_features,
        'Coefficient': np.abs(lasso.coef_)
    }).sort_values('Coefficient', ascending=False)
    
    results['lasso'] = lasso_importance
    
    # 3. Korrelationsanalyse
    correlation_matrix = X_numeric_imputed.corr()
    high_correlations = np.where(np.abs(correlation_matrix) > 0.7)
    high_corr_pairs = [(correlation_matrix.index[x], correlation_matrix.columns[y], 
                        correlation_matrix.iloc[x, y])
                       for x, y in zip(*high_correlations) 
                       if x != y and x < y]
    
    results['high_correlations'] = high_corr_pairs
    
    # Visualisierungen
    plt.figure(figsize=(12, 6))
    sns.barplot(data=f_scores.head(10), x='F_Score', y='Feature')
    plt.title(f'Top 10 Features nach F-Score für {target_name}')
    plt.tight_layout()
    plt.show()
    
    return results

In [ ]:
# Daten ohne Live-Content filtern
X_train_filtered = X_train[X_train['theme_Live'] == 0].copy()
y_train_filtered = y_train.loc[X_train_filtered.index]

print("Ursprüngliche Datengröße:", len(X_train))
print("Gefilterte Datengröße:", len(X_train_filtered))

# Feature-Analyse mit gefilterten Daten
results = analyze_features_for_target(X_train_filtered, y_train_filtered, 'aufrufe', feature_columns)

# Ausgabe der Ergebnisse
print("\nTop 10 Features nach F-Score:")
print(results['f_scores'].head(10))

print("\nTop 10 Features nach Lasso-Koeffizienten:")
print(results['lasso'].head(10))

print("\nStark korrelierte Features:")
for pair in results['high_correlations']:
    print(f"{pair[0]} -- {pair[1]}: {pair[2]:.3f}")

# Korrelation der Titelbewertung mit Aufrufen
correlation = X_train_filtered['Bewertung_Titel'].corr(y_train_filtered['aufrufe'])
print("\nKorrelation Titelbewertung mit Aufrufen:", correlation)

Analyse Kennzahlen wie F-Test oder p-Wert zwischen Aufrufen und BEwertung_Titel

In [ ]:
# Spezifische Analyse der Thumbnail-Bewertung
print("Position der Bewertung_Titel in den F-Scores:")
thumbnail_f_score = results['f_scores'][results['f_scores']['Feature'] == 'Bewertung_Titel']
print(thumbnail_f_score)

print("\nPosition der Bewertung_Titel in den Lasso-Koeffizienten:")
thumbnail_lasso = results['lasso'][results['lasso']['Feature'] == 'Bewertung_Titel']
print(thumbnail_lasso)

# Korrelation mit Aufrufen
correlation = X_train_filtered['Bewertung_Titel'].corr(y_train_filtered['aufrufe'])
print("\nKorrelation Bewertung_Titel mit Aufrufen:", correlation)

# Zusätzliche Visualisierung
plt.figure(figsize=(10, 6))
plt.scatter(X_train_filtered['Bewertung_Titel'], y_train_filtered['aufrufe'], alpha=0.5)
plt.xlabel('Bewertung_Titel (Bewertung)')
plt.ylabel('Aufrufe')
plt.title('Zusammenhang zwischen Bewertung_Titel und Aufrufen')
plt.show()

In [ ]:
# Daten ohne Live-Content filtern
X_train_filtered = X_train[X_train['theme_Live'] == 0].copy()
y_train_filtered = y_train.loc[X_train_filtered.index]

print("Ursprüngliche Datengröße:", len(X_train))
print("Gefilterte Datengröße:", len(X_train_filtered))

# Prüfen des Datentyps der Klickrate
print("\nDatentyp der Klickrate:")
print(y_train_filtered['Klickrate der Impressionen (%)'].dtype)

# Konvertierung der Klickrate in numerischen Typ
klickrate = pd.to_numeric(y_train_filtered['Klickrate der Impressionen (%)'].str.replace(',', '.').str.replace('%', ''), errors='coerce')

# Temporäres DataFrame für die Analyse
y_train_temp = pd.DataFrame({'klickrate': klickrate})

# Analyse der Klickrate
results_ctr = analyze_features_for_target(X_train_filtered, y_train_temp, 'klickrate', feature_columns)

print("\nTop Features für Klickrate:")
print(results_ctr['f_scores'].head(10))

# Spezifische Analyse Thumbnail/Titel auf Klickrate
correlation_thumbnail = X_train_filtered['Gestaltung_Thumbnail'].corr(klickrate)
correlation_titel = X_train_filtered['Bewertung_Titel'].corr(klickrate)

print("\nKorrelationen mit Klickrate:")
print(f"Thumbnail-Gestaltung: {correlation_thumbnail}")
print(f"Titel-Bewertung: {correlation_titel}")

Aufgrund der anderen Analysen scheinen mir die Prädiktoren auf die man Einfluss hat, keine wirkliche Rolle zu spielen. Den Titel kann ich beeinflussen und auch die Gestaltung der Thumbnails und natürlich die Länge der Videos. Das scheint mir nicht sinnvoll. Jeder weiß, dass der richtige Titel und vor allem ein gut gestaltetes Thumbnail entscheidende Faktoren sind

1. **Mögliche Gründe für die scheinbar geringen Effekte:**

**Bewertungssystem könnte zu grob sein:**
   - Die dreistufige Bewertung der Thumbnails (1,2,3) könnte zu undifferenziert sein
   - Die Komplexität guter Thumbnail-Gestaltung (Bildkomposition, Farbwahl, Textplatzierung, etc.) lässt sich möglicherweise nicht in drei Kategorien erfassen
   - Ähnliches gilt für die Titelbewertung durch die KI

**Zeitliche Dimension fehlt:**
   - Wir sehen nur die Endergebnisse der Videos
   - Die initiale Anziehungskraft von Thumbnail und Titel (die ersten Stunden/Tage) könnte in den Gesamtdaten untergehen
   - Der "First-Click-Effect" wird möglicherweise nicht richtig erfasst

**Indirekte Effekte werden nicht erfasst:**
   - Ein gutes Thumbnail führt zu mehr Klicks -> mehr Engagement -> bessere algorithmische Verbreitung
   - Diese Kausalkette wird in der einfachen Korrelationsanalyse nicht sichtbar

2. **Vorschlag für eine verfeinerte Analyse:**

**Zeitliche Analyse:**
   - Untersuchung der ersten 24/48 Stunden nach Veröffentlichung
   - Vergleich der Initial-Performance (wo Thumbnail/Titel den größten Einfluss haben sollten)

**Detailliertere Thumbnail-Bewertung:**
   - Mehr Kategorien (z.B. 5 oder 7 Stufen)
   - Separate Bewertung verschiedener Aspekte (Gesichter, Text, Kontrast, etc.)

**Klickraten-Analyse:**
   - Fokus auf das Verhältnis Impressionen/Klicks
   - Dies sollte den direkten Einfluss von Thumbnail und Titel besser zeigen

3. **Methodische Anpassung:**
   - Entwicklung eines komplexeren Bewertungssystems für Thumbnails
   - Berücksichtigung von Interaktionseffekten zwischen Titel und Thumbnail
   - Einbeziehung von zeitbasierten Metriken

